In [1]:
import tensorflow as tf
import numpy as np

In [14]:
X = []
t = 1
for i in range(t):
    with open("heavy_matrices/view" + str(i) + ".npy", 'rb') as f:
        X.append(tf.cast(tf.sparse.from_dense(np.load(f)), dtype=tf.float32))

In [15]:
k = 7
steps = 1
n = X[0].shape[1]
alpha, beta, gamma = 1.0, 1000, 0.01
wv = [1.0 / t for i in range(t)]
reg = gamma / (4.0 * beta)
I = tf.eye(n, dtype=tf.float32)
S = tf.eye(n, dtype=tf.float32)

In [4]:
for step in range(steps):
    S = tf.where(S > 0, S, tf.zeros(S.shape))
    S = (S + tf.transpose(S)) / 2.0
    acc = tf.zeros(S.shape)
    for i in range(t):
        Xv = tf.sparse.to_dense(X[i])
        aux = tf.linalg.matmul(Xv, Xv, transpose_a=True)
        Z = tf.linalg.matmul(
                tf.linalg.inv(aux + I * alpha + I * beta * wv[i]),
                S * beta * wv[i] + aux)
        Z = tf.where(Z[i] > 0, Z, tf.zeros(Z.shape))
        Z = (Z + tf.transpose(Z)) / 2.0
        wv[i] = 0.5 / tf.norm(Z - S)
        Z *= wv[i]
        acc += Z

    L = tf.linalg.diag(tf.math.reduce_sum(S, 0)) - S
    (_, v) = tf.linalg.eigh(L)

    p = tf.norm(v[0][:] - v, axis=1, keepdims=True)
    for i in range(1, n):
         p = tf.concat([p, tf.norm(v[i][:] - v, axis=1, keepdims=True)], 1)
    p = tf.math.square(p) * reg
    tmp = S
    S = acc - p / wv
    if step > 5 and tf.norm(S - tmp) / tf.norm(tmp) < 1.e-3:
        break
print(S)

Instructions for updating:
Use `tf.linalg.matmul` instead
tf.Tensor(
[[7.3236302e-03 4.0373369e-08 4.0397353e-08 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.0373369e-08 7.3236069e-03 1.3834533e-07 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.0397353e-08 1.3834533e-07 7.3236097e-03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]], shape=(4757, 4757), dtype=float32)


In [25]:
# x = tf.constant([[2., 16., 128.],
#                  [4., 32., 256.],
#                  [8., 64., 512.]], dtype=tf.float32)

False
